<a href="https://colab.research.google.com/github/stmulugheta/Introduction-to-Web-Scraping/blob/main/Carrefour_food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
#import sys
os.getcwd()

'/content'

In [ ]:
# Scraper for Carrefour Retailer Website in Belgium
import requests
from bs4 import BeautifulSoup as soup 
from urllib.request import urlopen as uReq
import pandas as pd
import re

In [ ]:
import time

In [ ]:
# The initial page of fresh food and drink
CarrefourURL = "https://drive.carrefour.be/nl/Verse-producten/c/ProduitsFrais?q=%3Arelevance"

In [ ]:
def has_numbers(inputString):
    # Check whether there is number in title
    return any(char.isdigit() for char in inputString)
     

In [ ]:
def html_parse(url):
  # Create a function to extract products variables from html 

  # Create a list to store all the products
  products_all = []
  # Create a variable to store page number
  page_no = 1

  # scrape products info page by page
  while True:
    
    # create a response object to get the web
    get_url = requests.get(url)
    data = get_url.text
    html_soup = soup(data, 'html.parser')

    # Create a list to store one page products
    products_one_page = []
    # Extract specific html tags into a list
    results = html_soup.find_all('div', {'class':'wrapper'})

      
    # create product dictionary to retain the attributes of each product
    for item in results:
        
        # Get product overall info
        product = item.find('a', {'name select_item name-title select_promotion_item'})
            
        # Get product id
        product_id = product.get('data-item_id')

        # Get brand
        try:
          brand = product.get('data-item_brand')
        except:
          brand = '-1'

        # Get product title, including product name and quantity
        title =  product.get('data-item_name')
     
        # Split title to name and quantity       
        title_split = title.split()

        # Identify quantity and title
        try: 
          quantity = item.find('span',{'class':'cart_dyanmic_quant'}).text.strip()[1:]
          # Partition title by '0.'
          product_name = title.partition('0.')[0]
        except:
          try:
            if has_numbers(title):           
              if title_split[-3] == 'x':   
                quantity = title_split[-4] + title_split[-3] + title_split[-2] + title_split[-1]
              else:
                quantity = title_split[-2] + title_split[-1]  
                # Identify name
                # Partition title by the first character of quantity     
                product_name = title.partition(quantity[0])[0]
              if quantity.replace(' ','').isalpha():
                quantity = '-1'
                product_name = title
              elif quantity[0].isalpha():
                split = re.split('(\d+)', quantity)
                quantity = ''.join(split[1:])
                product_name = product_name + ' ' + split[0]
          except:
                quantity = '-1'
                product_name = title

        try:
          if quantity == '-1':
            x = re.findall('\d\d\s', product_name)
            y = re.findall("\d\s", product_name)
            z = re.findall('(\d+)', product_name)
            if x:
              quantity = x[0]
            elif y:
              quantity = y[0] 
            elif z:
              quantity = ''.join(product_name.partition(z[0])[1:])
        except:
          quantity = '-1'



        # Get current price
        try:
          price = product.get('data-price')
        except:
          price = '-1'
        
        # Get original price
        # original_price equals to current price if no discount
        try:
          if product.get('data-base_price') == '':
            original_price = price
        except:
          original_price = '-1'

        # Get unnit price
        try:
          unit_price = item.find('span', {'class': 'bigprice'}).text.strip() + item.find('span', {'class': 'type'}).text.strip()
        except:
          unit_price = -1
        
        # Get promotion info
        try:
          promotion = item.find('input', {'class':'view_promotion'}).get('data-promotion_name')
        except: 
          promotion = '-1'

        # Get product link
        link = product.get('href')
        link = 'https://drive.carrefour.be' + link

        food = {
            'product_id': product_id,
            'brand': brand,
            'product_name': product_name,
            'quantity': quantity,
            'price': price, # current price
            'original_price': original_price,
            'unit_price': unit_price,
            'promotion': promotion,
            'link':link
        }
        
        products_one_page.append(food)
    
    # Link to next page
    url_tag = html_soup.find('li',{'class':'pagination-next'})
    
    # if page_no < 4: 
    if url_tag.find('a').get('href') != '#':
        url= 'https://drive.carrefour.be' + url_tag.find('a').get('href')
        products_all += products_one_page       
        page_no += 1
    else:
        products_all += products_one_page
        break
        
  print("There're totally {} pages, {} products.".format(page_no, len(products_all)))    
  return products_all
  


        

In [ ]:
t0 = time.time()
print(t0)
Carrefour_food = html_parse(CarrefourURL) 
t1 = time.time()
total_time = t1-t0
print(total_time)

1636437623.717057


AttributeError: ignored

In [ ]:
def output(productslist):
    # Convert the products list into a pandas dataframe and save as a csv file
    productsdf = pd.DataFrame(productslist)
    productsdf.to_csv('Carrefour_food.csv', index = False) #set index to false to avoid the pandas indexing
    print('Saved to CSV')
    return


In [ ]:
CarrefourProductsList = output(Carrefour_food)

Saved to CSV
